In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [ ]:
def fix_JSON(json_message=None):
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(e.message.split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ' '
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [ ]:
def clean_text(textdata):
    soup = BeautifulSoup(textdata)
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return (text.encode('utf-8'))

In [9]:
explainers = BeautifulSoup(requests.get("http://theconversation.com/au/factcheck").text, "lxml")
headlines = explainers.find_all('h2')
results = list()

In [10]:
for headline in headlines:
    thisline = dict()
    try:
        link = "http://theconversation.com" + headline.a['href']
        thisline['url'] = link
        thisline['headline'] = headline.text
        thisline['content'] = requests.get(link).text
        soup = BeautifulSoup(thisline['content'], "lxml")
        thisline['link'] = link
        meta = soup.find(attrs={'name' : "news_keywords"})
        thisline['keywords'] = meta['content']
        thisline['source'] = "The Conversation"
    except Exception as e:
        print("That went horribly wrong")
        print(e)
    results.append(thisline)

That went horribly wrong
'NoneType' object is not subscriptable
That went horribly wrong
'NoneType' object is not subscriptable


In [11]:
abcexplainers = BeautifulSoup(requests.get('http://www.abc.net.au/news/factcheck/zombies/').text, "lxml")

headlines = abcexplainers.find_all("a")

In [12]:
headlines = [headline for headline in headlines if "Fact Check zombie" in headline.text]

In [13]:
headlines[0].text

'Fact Check zombie: Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST'

In [15]:
for headline in headlines:
    thisline = dict()
    try:
        link = "http://abc.net.au" + headline['href']
        thisline=dict()
        thisline['url'] = link
        thisline['headline'] = headline.text
        thisline['content'] = requests.get(link).text
        soup = BeautifulSoup(thisline['content'], "lxml")
        thisline['link'] = link
        meta = soup.find(attrs={'name' : "keywords"})
        thisline['keywords'] = meta['content']
        thisline['source'] = "ABC"
    except:
        print("That went horribly wrong")
    results.append(thisline)

In [16]:
df = pd.DataFrame(results)

In [17]:
df = df[['link','source','keywords','headline','content']]

In [18]:
df

,link,source,keywords,headline,content
0,http://theconversation.com/election-factcheck-...,The Conversation,"Asylum seekers, Election factcheck, Election f...",Election FactCheck: Does the government spend ...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...
1,http://theconversation.com/election-factcheck-...,The Conversation,"Coalition, Election factcheck 2016, Federal el...",Has the Coalition invested an average of $5 bi...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...
2,http://theconversation.com/election-factcheck-...,The Conversation,"Diagnostic tests, Election factcheck 2016, Hea...",Has the Coalition cut bulk-billing for patholo...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...
3,http://theconversation.com/election-factcheck-...,The Conversation,"Election factcheck, Employment, Federal electi...","Have 300,000 new jobs been created in the last...",<!DOCTYPE html>\n<!--[if IE 7]> <html class...
4,http://theconversation.com/election-factcheck-...,The Conversation,Australian Charities and not-for-profit Commis...,Is the Australian Sex Party right about religi...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...
5,http://theconversation.com/election-factcheck-...,The Conversation,"Capital gains tax, Capital gains tax concessio...",Does the government spend more on negative gea...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...
6,http://theconversation.com/election-factcheck-...,The Conversation,"Climate change, Climate science, Election fact...","Are larger, more frequent storms predicted due...",<!DOCTYPE html>\n<!--[if IE 7]> <html class...
7,http://theconversation.com/election-factcheck-...,The Conversation,"Bill Shorten, Election factcheck 2016, Employm...","Have 50,000 full-time jobs been lost this year...",<!DOCTYPE html>\n<!--[if IE 7]> <html class...
8,http://theconversation.com/election-factcheck-...,The Conversation,"Election factcheck 2016, Federal election 2016...",Is it true no solely managed Commonwealth fish...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...
9,http://theconversation.com/election-factcheck-...,The Conversation,"Bill Shorten, Debt, Election factcheck 2016, F...",Has $100 billion been added to Australia’s nat...,<!DOCTYPE html>\n<!--[if IE 7]> <html class...


In [19]:
df['content'] = df['content'].fillna(" ")

In [20]:
df['content'] = df['content'].apply(clean_text)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [21]:
df

,link,source,keywords,headline,content
0,http://theconversation.com/election-factcheck-...,The Conversation,"Asylum seekers, Election factcheck, Election f...",Election FactCheck: Does the government spend ...,"b""Election FactCheck: Does the government spen..."
1,http://theconversation.com/election-factcheck-...,The Conversation,"Coalition, Election factcheck 2016, Federal el...",Has the Coalition invested an average of $5 bi...,"b""Election FactCheck: Has the Coalition invest..."
2,http://theconversation.com/election-factcheck-...,The Conversation,"Diagnostic tests, Election factcheck 2016, Hea...",Has the Coalition cut bulk-billing for patholo...,"b""Election FactCheck: has the Coalition cut bu..."
3,http://theconversation.com/election-factcheck-...,The Conversation,"Election factcheck, Employment, Federal electi...","Have 300,000 new jobs been created in the last...","b'Election FactCheck: Have 300,000 new jobs be..."
4,http://theconversation.com/election-factcheck-...,The Conversation,Australian Charities and not-for-profit Commis...,Is the Australian Sex Party right about religi...,"b""Election FactCheck: is the Australian Sex Pa..."
5,http://theconversation.com/election-factcheck-...,The Conversation,"Capital gains tax, Capital gains tax concessio...",Does the government spend more on negative gea...,"b""Election FactCheck Q&A: does the government ..."
6,http://theconversation.com/election-factcheck-...,The Conversation,"Climate change, Climate science, Election fact...","Are larger, more frequent storms predicted due...","b'Election FactCheck: are larger, more frequen..."
7,http://theconversation.com/election-factcheck-...,The Conversation,"Bill Shorten, Election factcheck 2016, Employm...","Have 50,000 full-time jobs been lost this year...","b""Election FactCheck: have 50,000 full-time jo..."
8,http://theconversation.com/election-factcheck-...,The Conversation,"Election factcheck 2016, Federal election 2016...",Is it true no solely managed Commonwealth fish...,"b""Election FactCheck: is it true no solely man..."
9,http://theconversation.com/election-factcheck-...,The Conversation,"Bill Shorten, Debt, Election factcheck 2016, F...",Has $100 billion been added to Australia’s nat...,"b""Election FactCheck: has $100 billion been ad..."


In [22]:
df.to_pickle('conversationexplainers.pickle')

In [ ]:
soup = BeautifulSoup(somecontent)